In [1]:
import openai

openai_api_key = 'sk-'
openai.api_key = openai_api_key

In [5]:

from typing import Optional


example_completion="Hey Chris! I read your profile and I wanted to send a quick note to say hello. I also wanted to let you know that I've heard of Levels.fyi - I think your company is really interesting and I have to say I'm a bit jealous that you're in charge of comp & benefits at Reddit!  "
example_fix="The completion needs to ask Chris if they have heard about Levels.fyi, not share that 'I've heard of Levels.fyi.'"
example_completion2="I've really been impressed after reading up on your work at Self Financial. It seems like you're leading the growth marketing efforts which is amazing given the context. I'm curious to know more about how you're leading the charge given your impressive background with other awesome brands.  "
example_fix2="Remove references to hardship, specifically 'given the context.'"


def wrapped_chat_gpt_completion(
    messages: list,
    history: Optional[list] = [],
    max_tokens: Optional[int] = 100,
    temperature: Optional[float] = 0.75,
    top_p: Optional[float] = 1,
    n: Optional[int] = 1,
    frequency_penalty: Optional[float] = 0,
):
    if history:
        messages = history + messages

    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature,
        top_p=top_p,
        n=n,
        frequency_penalty=frequency_penalty,
    )
    if response is None or response["choices"] is None or len(response["choices"]) == 0:
        return [], ""

    choices = response["choices"]
    top_choice = choices[0]
    preview = top_choice["message"]["content"].strip()

    messages = messages + [{"role": "assistant", "content": preview}]
    return messages, preview


def preview_fix(completion: str, fix: str):
    """ Previews the fix for a given completion.

    Args:
        completion (str): Completion to preview the fix for.
        fix (str): Fix to preview.

    Returns:
        str: Preview of the fix.
    """
    # Define the maximum number of generated tokens to be equal to 1.25 the original message.
    max_tokens_length = int(len(completion) * 1.25)
    completion = completion.strip()
    fix = fix.strip()
    
    messages, preview = wrapped_chat_gpt_completion(
        messages=[
            {"role": "user", "content": "completion: {}\nfix: {}\ncompletion:".format(completion, fix)},
        ],
        max_tokens=max_tokens_length,
        temperature=0,
    )

    return preview, 200

returned = preview_fix(example_completion2, example_fix2)
print(returned)

{'preview': "I've really been impressed after reading up on your work at Self Financial. It seems like you're leading the growth marketing efforts which is amazing. I'm curious to know more about how you're leading the charge given your impressive background with other awesome brands.", 'status': 200}
